* Indexing method for making dictionary keys
* Create full list of positions
* Function to convert index keys to simulation inputs
* MC simulation
* Exact analytical solution
* Algo for using simulation to solve for strategy
* Compare analytical and numerical solution
